# 1、寫一個均倉策略
# 2、如何debug
# 3、如何使用FMZ（botvs）的回測

In [1]:
class mid_class():
    def __init__(self, this_exchange):
        '''
        初始化數據填充交易所的信息，首次獲取價格，首次獲取account信息
        設定好密鑰……
        
        Args:
            this_exchange: FMZ的交易所結構
        
        '''
        self.init_timestamp = time.time()
        self.exchange = this_exchange
        self.name = self.exchange.GetName()
        self.jyd = self.exchange.GetCurrency()        
    
    def get_account(self):
        '''
        獲取賬戶信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.Balance = '---'
        self.Amount = '---'
        self.FrozenBalance = '---'
        self.FrozenStocks = '---'
        
        try:
            self.account = self.exchange.GetAccount()

            self.Balance =  self.account['Balance']
            self.Amount = self.account['Stocks']
            self.FrozenBalance =  self.account['FrozenBalance']
            self.FrozenStocks = self.account['FrozenStocks']
            return True
        except:
            return False
    
    def get_ticker(self):
        '''
        獲取市價信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.high = '---'
        self.low = '---'
        self.Sell =  '---'
        self.Buy =  '---'
        self.last =  '---'
        self.Volume = '---'
        
        try:
            self.ticker = self.exchange.GetTicker()
        
            self.high = self.ticker['High']
            self.low = self.ticker['Low']
            self.Sell =  self.ticker['Sell']
            self.Buy =  self.ticker['Buy']
            self.last =  self.ticker['Last']
            self.Volume = self.ticker['Volume']
            return True
        except:
            return False
        
        
    def get_depth(self):
        '''
        獲取深度信息
        
        Returns:
            獲取信息成功返回True，獲取信息失敗返回False
        '''
        self.Ask = '---'
        self.Bids = '---'
        
        try:
            self.Depth = self.exchange.GetDepth()
            self.Ask = self.Depth['Asks']
            self.Bids = self.Depth ['Bids']
            return True
        except:
            return False
        
        
    
    def get_ohlc_data(self, period = PERIOD_M5):
        '''
        獲取K線信息
        
        Args:
            period: K線週期，PERIOD_M1 指1分鐘, PERIOD_M5 指5分鐘, PERIOD_M15 指15分鐘,
            PERIOD_M30 指30分鐘, PERIOD_H1 指1小時, PERIOD_D1 指一天。
        '''
        self.ohlc_data = exchange.GetRecords(period)
        
        
    
    def create_order(self, order_type, price, amount):
        '''
        post一個掛單信息
        
        Args:
            order_type：掛單類型，'buy'指掛買單，'sell'指掛賣單
            price：掛單價格
            amount:掛單數量
            
        Returns:
            掛單Id號，可用以取消掛單
        '''
        if order_type == 'buy':
            try:
                order_id = self.exchange.Buy( price, amount)
            except:
                return False
            
        elif order_type == 'sell':
            try:
                order_id = self.exchange.Sell( price, amount)
            except:
                return False
        
        return order_id
    
    def cancel_order(self, order_id):
        '''
        取消一個掛單信息
        
        Args:
            order_id：希望取消的掛單ID號
            
        Returns:
            取消掛單成功返回True，取消掛單失敗返回False
        '''
        return self.exchange.CancelOrder(order_id)
        
    def refreash_data(self):
        '''
        刷新信息
        
        Returns:
            刷新信息成功返回 'refreash_data_finish!' 否則返回相應刷新失敗的信息提示
        '''

        if not self.get_account():
            return 'false_get_account'
        
        if not self.get_ticker():
            return 'false_get_ticker'
        if not self.get_depth():
            return 'false_get_depth'
        try:
            self.get_ohlc_data()
        except:
            return 'false_get_K_line_info'
        
        return 'refreash_data_finish!'


In [3]:
class juncang_class():
    def __init__(self, mid_class):
        self.jys = mid_class
        self.last_time = time.time()
        self.last_trade_price = self.jys.last
        
    def make_need_account_info(self):
        self.jys.refreash_data()
        self.B = self.jys.Amount
        self.money = self.jys.Balance
        now_price = self.jys.last
        
        self.total_money = self.B* now_price + self.money
        self.half_money = self.total_money/2
        self.need_buy = (self.half_money - self.B*now_price )/now_price
        self.need_sell = (self.half_money - self.money)/now_price
    
    def do_juncang(self):
        if self.need_buy > 0.001:
            self.jys.create_order( 'buy', self.jys.low , self.need_buy ) 
        elif self.need_sell > 0.001:
            self.jys.create_order( 'sell', self.jys.high , self.need_sell ) 
        
    
    def if_need_trade(self, condition, prama):
        if condition == 'time':
            if time.time() - self.last_time > prama:
                self.do_juncang()
                self.last_time = time.time()
        if condition == 'price':
            if abs((self.jys.last -  self.last_trade_price)/self.last_trade_price)  > prama:
                self.do_juncang()
                self.last_trade_price = self.jys.last

In [4]:
def main():
    test_mid = mid_class(exchange)
    Log(test_mid.refreash_data())
    test_juncang = juncang_class(test_mid)
    
    while(True):
        Sleep(1000)
        test_juncang.make_need_account_info()
        test_juncang.if_need_trade('price', 0.05)

In [ ]:
1 1000 --- 1000

In [5]:
0.75 + 1500 ---- 2000

1.5

In [8]:
1.125 1125  --- 1000

1125